In [6]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import json
import pandas as pd
import numpy as np
import random

import warnings
warnings.filterwarnings('ignore')

In [7]:
URL = 'https://www.findamasters.com/masters-degrees/msc-degrees/usa/full-time/?430l-3h0&Keywords=%22data%22%20or%20%22analytics%22%20or%20%22analysis%22%20or%20%22machine%20learning%22%20or%20%22ml%22%20or%20%22artificial%20intelligence%22%20or%20%22statistics%22'
# obtained after searching with the relevant keywords

In [121]:
def extract_data(response):

    # make soup
    soup = BeautifulSoup(response.content, 'html.parser')

    # get data
    programs = soup.find_all('div', class_ = 'course-content-info')
    titles = [program.h3.text for program in programs]
    institutes = [program.select('.instLink')[0].text for program in programs]
    departments = [program.select('.deptLink')[0].text for program in programs]
    labels = [program.select('.course-icon-area')[0].text for program in programs] 
    labels = [label.replace('\xa0', '').replace('\n', ',').strip(',') for label in labels]

    return titles, institutes, departments, labels

def make_data(base_url, low, high):

    title = []
    inst = []
    dept = []
    lab = []

    for page_num in range(low, high+1):

        time.sleep(5)
        print(f'Extracting from page {page_num}')

        try:
            response = requests.get(
                base_url,
                params = {'PG':page_num},
                headers = {'User-agent': 'DSE Scraper'},
                timeout=(5,5)
            )
        except:
            print(f'Connection dropped. Page: {page_num}')
            continue

        t, i, d, l = extract_data(response)
        title += t
        inst += i
        dept += d
        lab += l

        print(len(t))

    df = pd.DataFrame(
        dict(
            title = title,
            institute = inst,
            department = dept,
            label = lab
        )
    )

    print('Data extraction complete.')

    return df        

In [122]:
temp_df = make_data(URL, 1, 9)

Extracting from page 1
15
Extracting from page 2
15
Extracting from page 3
15
Extracting from page 4
15
Extracting from page 5
15
Extracting from page 6
2
Extracting from page 7
0
Extracting from page 8
0
Extracting from page 9
0
Data extraction complete.


Problems  
1. From page 6, only 2 courses extracted  
2. From page 7, none were extracted  
3. From page 8, none were extracted  
4. From page 9, none were extracted

In [123]:
temp_df.shape

(77, 4)

In [125]:
temp_df['title'] = temp_df.title.apply(lambda x: x.replace('\n', ' ').strip())

In [128]:
temp_df.to_csv('data/findamasters_usa_raw_jun1_2021.csv', index=False)